# 🤖 Question Answering System Using BERT

This notebook implements an **extractive question answering system**
using a pre-trained **BERT Large** model fine-tuned on the **SQuAD dataset**.

The system answers user questions by extracting relevant information
directly from a fixed context describing a fictional automobile dealership.


In [ ]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

## Model and Tokenizer Setup

A BERT Large model fine-tuned for question answering is loaded along with
its corresponding tokenizer.

The model is capable of predicting the start and end positions of an answer
within a given context passage.


In [ ]:
model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad' 

In [ ]:
model =  BertForQuestionAnswering.from_pretrained(model_name)   #loading the pre-trained model

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_name) 

## Context Definition

A static context passage describing *Sunset Motors* is used as the
knowledge base for the question answering system.

All user questions are answered exclusively using information
contained within this context.


In [ ]:
sunset_motors_context = "Sunset Motors is a renowned automobile dealership that has been a cornerstone of the automotive industry since its establishment in 1978. Located in the picturesque town of Crestwood, nestled in the heart of California's scenic Central Valley, Sunset Motors has built a reputation for excellence, reliability, and customer satisfaction over the past four decades. Founded by visionary entrepreneur Robert Anderson, Sunset Motors began as a humble, family-owned business with a small lot of used cars. However, under Anderson's leadership and commitment to quality, it quickly evolved into a thriving dealership offering a wide range of vehicles from various manufacturers. Today, the dealership spans over 10 acres, showcasing a vast inventory of new and pre-owned cars, trucks, SUVs, and luxury vehicles. One of Sunset Motors' standout features is its dedication to sustainability. In 2010, the dealership made a landmark decision to incorporate environmentally friendly practices, including solar panels to power the facility, energy-efficient lighting, and a comprehensive recycling program. This commitment to eco-consciousness has earned Sunset Motors recognition as an industry leader in sustainable automotive retail. Sunset Motors proudly offers a diverse range of vehicles, including popular brands like Ford, Toyota, Honda, Chevrolet, and BMW, catering to a wide spectrum of tastes and preferences. In addition to its outstanding vehicle selection, Sunset Motors offers flexible financing options, allowing customers to secure affordable loans and leases with competitive interest rates."

## FAQ Bot Implementation

A function is defined to:
- Accept a natural language question
- Encode the question together with the context
- Construct segment (token type) IDs for BERT
- Run the model to obtain start and end logits
- Extract and reconstruct the answer text from tokens

The function also handles subword tokens produced by BERT
to return a clean, readable answer.


In [ ]:
def faq_bot(question):
    context = sunset_motors_context
    input_ids = tokenizer.encode(question, context)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    if answer_end >= answer_start:
        answer = ' '.join(tokens[answer_start: answer_end+1])
    else:
        print("I don't know the answer to that question.can you please rephrase it?")
    corrected_answer = ''
    for word in answer.split():
        if word[0:2] == '##':
            corrected_answer += word[2:]
        else:
            corrected_answer += ' ' + word
    return corrected_answer.strip()

## User Interaction

The system prompts the user for a question and returns an extracted
answer based on the provided context.

If the model cannot confidently identify a valid answer span,
the system requests the user to rephrase the question.


In [ ]:
user_input = input("Please enter your question about Sunset Motors: ")
print(faq_bot(user_input))

## Summary

This notebook demonstrates:
- Practical use of BERT for extractive question answering
- Manual construction of segment embeddings
- Token-level answer extraction and post-processing
- Interactive user-driven querying of a fixed knowledge base

The implementation highlights applied understanding of
Transformer-based QA systems and their real-world usage.
